### Setting up the connection

In [ ]:
!pip install xgboost==2.0.1
!pip install catboost==1.2.2


In [1]:
# Import Packages
import pymysql
import pandas as pd

In [2]:
# Using mymysql to connect to ai300 remote database, these are the credentials
ENDPOINT = 'heicoders-playground.c2ced10ceyki.ap-southeast-1.rds.amazonaws.com'
PORT = 3306
USERNAME = 'student300'
DBNAME = 'ai300_capstone'
PASSWORD = 'heicoders_AI300'   # Default password is empty. Do change this if you had previously set a password.
CURSORCLASS = pymysql.cursors.DictCursor

In [3]:
# This is the connection object
def initiate_local_connection():
    try:
        connection = pymysql.connect(host=ENDPOINT,
                                     port=PORT,
                                     user=USERNAME,
                                     passwd=PASSWORD,
                                     db=DBNAME,
                                     cursorclass=CURSORCLASS)
        print('[+] Local Connection Successful')
    except Exception as e:
        print(f'[+] Local Connection Failed: {e}')
        connection = None

    return connection

In [4]:
connection = initiate_local_connection()

[+] Local Connection Successful


In [5]:
sql_query = f'SELECT * FROM ai300_capstone.account;'

def get_records(sql_query):
    try:
        with connection.cursor() as cursor:
            cursor.execute(sql_query)

        # Connection is not autocommit by default, so we must commit to save changes
        connection.commit()
        
        # Fetch all the records from SQL query output
        results = cursor.fetchall()
        
        # Convert results into pandas dataframe
        df = pd.DataFrame(results)
        
        print(f'Successfully retrieved records')
        
        return df
        
    except Exception as e:
        print(f'Error encountered: {e}')

### SQL queries to retrieve information from remote database

In [6]:
sql_query = '''
        SELECT * 
        FROM churn_status cs
        INNER JOIN customer c ON cs.customer_id = c.customer_id 
        INNER JOIN account a ON cs.customer_id = a.customer_id
        INNER JOIN city ci ON c.zip_code = ci.zip_code
        INNER JOIN account_usage au ON a.account_id = au.account_id;
'''

combined_df = get_records(sql_query)
combined_df

Successfully retrieved records


,customer_id,status,churn_label,churn_category,churn_reason,c.customer_id,gender,age,senior_citizen,married,...,au.account_id,avg_long_distance_fee_monthly,total_long_distance_fee,avg_gb_download_monthly,stream_tv,stream_movie,stream_music,total_monthly_fee,total_charges_quarter,total_refunds
0,0013-EXCHZ,Churned,Yes,Dissatisfaction,Network reliability,0013-EXCHZ,Female,75,Yes,Yes,...,BFIN-DLMOA,7.38,22.14,11,Yes,No,No,83.90,267.40,0.00
1,0014-BMAQU,Stayed,No,,,0014-BMAQU,Male,52,No,Yes,...,AFEO-XOOCP,12.96,816.48,7,No,No,No,84.65,5377.80,0.00
2,0016-QLJIS,Stayed,No,,,0016-QLJIS,Female,43,No,Yes,...,DEMQ-MFXWC,28.46,1849.90,14,Yes,Yes,Yes,90.45,5957.90,0.00
3,0019-EFAEP,Stayed,No,,,0019-EFAEP,Female,32,No,No,...,AIPP-VTDXJ,2.25,162.00,16,Yes,No,No,101.30,7261.25,0.00
4,0019-GFNTW,Stayed,No,,,0019-GFNTW,Female,39,No,No,...,CJHA-SRKIB,0.00,0.00,19,No,No,No,45.05,2560.10,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9412-ARGBX,Churned,Yes,Competitor,Competitor made better offer,9412-ARGBX,Female,47,No,No,...,XFNW-OGLLY,29.54,1417.92,29,Yes,No,No,95.50,4627.85,0.00
7039,9522-ZSINC,Stayed,No,,,9522-ZSINC,Male,34,No,No,...,SVZS-SEWJU,7.33,95.29,0,No,No,No,19.95,253.80,0.00
7040,9524-EGPJC,Churned,Yes,Competitor,Competitor offered higher download speeds,9524-EGPJC,Female,33,No,No,...,SENC-XOATR,34.21,615.78,19,No,Yes,Yes,90.10,1612.75,48.26
7041,9975-SKRNR,Joined,No,,,9975-SKRNR,Male,24,No,No,...,CYDP-LCTZB,49.51,49.51,0,No,No,No,18.90,18.90,0.00


In [ ]:
sql_query = f'SELECT * FROM ai300_capstone.account;'

df1 = get_records(sql_query)
df1

In [ ]:
sql_query = f'SELECT * FROM ai300_capstone.account_usage;'

df2 = get_records(sql_query)
df2

In [ ]:
sql_query = f'SELECT * FROM ai300_capstone.churn_status;'

df3 = get_records(sql_query)
df3

In [ ]:
sql_query = f'SELECT * FROM ai300_capstone.city;'

df4 = get_records(sql_query)
df4

In [ ]:
sql_query = f'SELECT * FROM ai300_capstone.customer;'

df5 = get_records(sql_query)
df5

### Dropping all repeated columns from the combined dataframe

In [ ]:
combined_df = combined_df.drop(
    columns=['c.customer_id','a.customer_id','ci.zip_code','au.account_id']
)

combined_df

### EDA 
#### Currently there are df1 to df5, each representing an extracted table, while combined_df is all data in 1 table

In [ ]:
# since all tables have been combined together
pd.options.display.max_columns = 50

In [ ]:
combined_df.head()

# target parameter is churn label

In [ ]:
combined_df.info()

In [ ]:
combined_df.nunique()

In [ ]:
df1.info()
print()
df2.info()
print()
df3.info()
print()
df4.info()
print()
df5.info()
print()

In [ ]:
df1.nunique()

In [ ]:
df5.nunique()

In [ ]:
import plotly.express as px

corr = combined_df.corr()
fig = px.imshow(corr, color_continuous_scale = 'Brwnyl', text_auto = True)
fig.update_xaxes(side="top")
fig.show()


### Preliminary Visualizations

In [ ]:
import matplotlib.pyplot as plt 
import plotly.express as px

In [ ]:
fig = px.scatter_matrix(combined_df, dimensions=['zip_code', 'city', 'latitutde','longitude','population'],
                       color='churn_label')


fig.show()


In [ ]:
fig = px.scatter_matrix(combined_df, dimensions=['gender', 'age', 'senior_citizen','married','num_dependents'],
                       color='churn_label')


fig.show()


In [ ]:
fig = px.scatter_matrix(combined_df, 
                        dimensions=['avg_long_distance_fee_monthly', 'total_long_distance_fee', 
                                    'avg_gb_download_monthly','stream_tv','stream_movie','stream_music'
                                   ,'total_monthly_fee','total_charges_quarter','total_refunds'],
                       color='churn_label')


fig.show()

In [ ]:
fig = px.scatter_matrix(combined_df, 
                        dimensions=['avg_long_distance_fee_monthly', 'total_long_distance_fee',
                                    'total_charges_quarter','avg_gb_download_monthly' 
                                  ],
                       color='churn_label')


fig.show()

In [ ]:
fig = px.scatter_matrix(combined_df, 
                        dimensions=['tenure_months', 'num_referrals', 
                                    'has_internet_service','internet_type','has_unlimited_data','has_phone_service',
                                   'has_multiple_lines','has_premium_tech_support','has_online_security',
                                   'has_online_backup','has_device_protection','contract_type','paperless_billing',
                                   'payment_method'],
                       color='churn_label')


fig.show()

In [ ]:
fig = px.scatter_matrix(combined_df, 
                        dimensions=['tenure_months', 'num_referrals', 
                                    'internet_type','contract_type',
                                   'payment_method'],
                       color='churn_label')


fig.show()

In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='num_referrals',
                    height = 400)


fig.show()

#num_referrals 0 has a larger portion

In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='internet_type',
                    height = 400)


fig.show()

#internet type fiber optic is a bigger portion

In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='contract_type',
                    height = 400)


fig.show()

#contract_type 0 has a larger portion of leavers

In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='population',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='tenure_months',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='has_internet_service',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='internet_type',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='has_unlimited_data',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='has_phone_service',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='has_multiple_lines',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='has_premium_tech_support',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='has_online_security',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='has_online_backup',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='has_device_protection',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='paperless_billing',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='payment_method',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='stream_tv',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='stream_movie',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='stream_music',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='gender',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='age',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='senior_citizen',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='married',
                    height = 400)


fig.show()


In [ ]:
fig = px.histogram(combined_df,  x='churn_label',
                    color='num_dependents',
                    height = 400)


fig.show()


In [ ]:
fig = px.bar(combined_df['payment_method'].value_counts()) 


fig.show()

In [ ]:
fig = px.bar(combined_df['churn_label'].value_counts()) 
            
fig.show()
# From dataset, how many customers have churned in the last month

In [ ]:
yes_churn_df = combined_df[combined_df.churn_label == "Yes"]

fig = px.bar(yes_churn_df['churn_category'].value_counts()) 
            
fig.show()
# From those that have churned, what is the main cause? Competitor

### Based on the above, features chosen  are 
### has_premium_tech_support, contract_type, internet_type, has_unlimited_data, num_dependents, num_referals

### Quick EDA of the new DF to decide on how to engineer the features

In [ ]:
combined_df['num_dependents']

In [ ]:
combined_df['churn_label'].unique()

In [7]:
selected_df = combined_df[['has_premium_tech_support','contract_type','internet_type',
             'has_unlimited_data','num_dependents', 'num_referrals',
             'churn_label']]

selected_df

,has_premium_tech_support,contract_type,internet_type,has_unlimited_data,num_dependents,num_referrals,churn_label
0,Yes,Month-to-Month,Fiber Optic,Yes,0,3,Yes
1,Yes,Two Year,Fiber Optic,No,0,8,No
2,Yes,Two Year,Cable,Yes,1,3,No
3,No,Two Year,Fiber Optic,Yes,0,0,No
4,Yes,Two Year,DSL,Yes,0,0,No
...,...,...,...,...,...,...,...
7038,Yes,Two Year,Fiber Optic,Yes,0,0,Yes
7039,No,One Year,None,No,0,0,No
7040,Yes,Month-to-Month,Fiber Optic,Yes,0,0,Yes
7041,No,Month-to-Month,None,No,0,0,No


In [ ]:
selected_df.nunique()

In [ ]:
selected_df['has_unlimited_data'].unique()

# to check out the values within the values all columns

In [ ]:
selected_df.info()

# no missing data found, most of the columns are categorical data

### Feature Engineering

In [8]:

# replacing categorical data denoted by strings in each column to numerical values
selected_df['has_premium_tech_support'].replace(['Yes','No'],[1,0], inplace=True) 
selected_df['contract_type'].replace(['Month-to-Month', 'Two Year', 'One Year'],[0,1,2], inplace=True) 
selected_df['internet_type'].replace(['Fiber Optic', 'Cable', 'DSL', 'None'],[1,2,3,0], inplace=True) 
selected_df['has_unlimited_data'].replace(['Yes','No'],[1,0], inplace=True)

selected_df['churn_label'].replace(['Yes','No',''],[1,0,0], inplace=True)
selected_df




/var/folders/js/fwrm2kcs5993hjm2q5q3kcqw0000gn/T/ipykernel_79956/3313127059.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['has_premium_tech_support'].replace(['Yes','No'],[1,0], inplace=True)
/var/folders/js/fwrm2kcs5993hjm2q5q3kcqw0000gn/T/ipykernel_79956/3313127059.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['contract_type'].replace(['Month-to-Month', 'Two Year', 'One Year'],[0,1,2], inplace=True)
/var/folders/js/fwrm2kcs5993hjm2q5q3kcqw0000gn/T/ipykernel_79956/3313127059.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cav

,has_premium_tech_support,contract_type,internet_type,has_unlimited_data,num_dependents,num_referrals,churn_label
0,1,0,1,1,0,3,1
1,1,1,1,0,0,8,0
2,1,1,2,1,1,3,0
3,0,1,1,1,0,0,0
4,1,1,3,1,0,0,0
...,...,...,...,...,...,...,...
7038,1,1,1,1,0,0,1
7039,0,2,0,0,0,0,0
7040,1,0,1,1,0,0,1
7041,0,0,0,0,0,0,0


### Testing out xgb as 1st model

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statistics

features = ['has_premium_tech_support', 'contract_type', 'internet_type', 
            'has_unlimited_data', 'num_dependents','num_referrals']
X = selected_df[features]
y = selected_df['churn_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)


# Instantiate the model object
model = xgb.XGBClassifier(learning_rate=0.2, max_depth=2, min_child_weight=5, random_state=5)

# Fit the model with the training data
model.fit(X_train, y_train)

# predict the target on the test dataset
y_predict = model.predict(X_test)
print('\nPrediction on test data', y_predict) 

# Accuracy Score on test dataset
accuracy_test = metrics.accuracy_score(y_test, y_predict)
print('\nAccuracy_score on test dataset : ', accuracy_test)

In [ ]:
from catboost import CatBoostClassifier 
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statistics

features = ['has_premium_tech_support', 'contract_type', 'internet_type', 
            'has_unlimited_data', 'num_dependents','num_referrals']
X = selected_df[features]
y = selected_df['churn_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)


# Instantiate the model object
model = CatBoostClassifier(learning_rate=0.1, depth=1, random_seed=5)

# Fit the model with the training data
model.fit(X_train, y_train)

# predict the target on the test dataset
y_predict = model.predict(X_test)
print('\nPrediction on test data', y_predict) 

# Accuracy Score on test dataset
accuracy_test = metrics.accuracy_score(y_test, y_predict)
print('\nAccuracy_score on test dataset : ', accuracy_test)

### Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

# this was performed for xgBoost
param_grid = {
    'max_depth': [1,2,3,4,5],
    'min_child_weight': [1,2,3,4,5],
    'eta': [0.1, 0.3, 0.5]
}

grid_search = GridSearchCV(estimator=xgb.XGBClassifier(), param_grid=param_grid, cv=10)
grid_search.fit(X_train, y_train)


In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'depth': [1,2,3,4,5],
    'learning_rate': [0.1]
}

grid_search = GridSearchCV(estimator= CatBoostClassifier(), param_grid=param_grid, cv=10)
grid_search.fit(X_train, y_train)

In [ ]:
# Get the best estimator
best_estimator = grid_search.best_estimator_

# Get the best hyperparameters
best_params = grid_search.best_params_

print(best_estimator)
print()
print(best_params)

In [9]:
from catboost import CatBoostClassifier 
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statistics

features = ['has_premium_tech_support', 'contract_type', 'internet_type', 
            'has_unlimited_data', 'num_dependents','num_referrals']
X = selected_df[features]
y = selected_df['churn_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)


# Instantiate the model object
model = CatBoostClassifier(learning_rate=0.1, depth=2, random_seed=5)

# Fit the model with the training data
model.fit(X_train, y_train)

# predict the target on the test dataset
y_predict = model.predict(X_test)
print('\nPrediction on test data', y_predict) 

# Accuracy Score on test dataset
accuracy_test = metrics.accuracy_score(y_test, y_predict)
print('\nAccuracy_score on test dataset : ', accuracy_test)

0:	learn: 0.6352847	total: 63.1ms	remaining: 1m 3s
1:	learn: 0.5813143	total: 65.1ms	remaining: 32.5s
2:	learn: 0.5419726	total: 66.4ms	remaining: 22.1s
3:	learn: 0.5141175	total: 67.9ms	remaining: 16.9s
4:	learn: 0.4919181	total: 69.3ms	remaining: 13.8s
5:	learn: 0.4805249	total: 70.7ms	remaining: 11.7s
6:	learn: 0.4658242	total: 71.9ms	remaining: 10.2s
7:	learn: 0.4503972	total: 73.8ms	remaining: 9.15s
8:	learn: 0.4440932	total: 75.1ms	remaining: 8.27s
9:	learn: 0.4365259	total: 76.5ms	remaining: 7.57s
10:	learn: 0.4311280	total: 78.1ms	remaining: 7.02s
11:	learn: 0.4238953	total: 80ms	remaining: 6.58s
12:	learn: 0.4200252	total: 81.4ms	remaining: 6.18s
13:	learn: 0.4167030	total: 82.7ms	remaining: 5.82s
14:	learn: 0.4137210	total: 84.2ms	remaining: 5.53s
15:	learn: 0.4106206	total: 85.6ms	remaining: 5.26s
16:	learn: 0.4081552	total: 87ms	remaining: 5.03s
17:	learn: 0.4050558	total: 88.4ms	remaining: 4.82s
18:	learn: 0.4034473	total: 89.9ms	remaining: 4.64s
19:	learn: 0.4002515	total

Exporting Model

In [10]:
from pathlib import Path

Path("../model").mkdir(exist_ok=True)

In [11]:
import joblib

joblib.dump(model, '../model/catboost_model.pkl')

['../model/catboost_model.pkl']